In [6]:
import pandas as pd
import numpy as np
import pyranges
import pickle
import os.path

In [7]:
hicLoops = pd.read_csv('./Dataset/merged_loops.bedpe',delimiter='\t')
ctcfPeaks = pyranges.read_bed('./Dataset/ENCFF356LIU.bed').df
chiaPetLoops = pd.read_csv('./Dataset/4DNFIS9CCN6R.bedpe',delimiter='\t', header=None)
rad21Peaks=pyranges.read_bed('./Dataset/ENCFF834GOT.bed').df

hicLoops=hicLoops[1:]
chiaPetLoops.columns = ['#chr1','x1','x2','chr2','y1','y2','?']

c:\Users\prusak.patryk\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyranges\methods\init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
c:\Users\prusak.patryk\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyranges\methods\init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


In [8]:
ctcfPeaks['Chromosome']=ctcfPeaks['Chromosome'].apply(lambda x: x.removeprefix('chr'))

1. Given a loop, consider chr1, x1, x2, chr2, y1, y2
2. For each row check whether a matching peak has an anchor in chr1 or chr2 or both
3. Could be done with 2 for loops but lets try to get it dont maybe with df?

In [4]:
def getAnchors(loops,peaks):
    bothAnchorsIndexes = []
    singleAnchorCount=0
    for i,loop in loops.iterrows():
        anchor1=False
        anchor2=False
        sameChromosome = peaks[peaks['Chromosome']==loop['#chr1']]
        x1=sameChromosome[sameChromosome['Start']>=loop['x1']]
        x2=x1[x1['End']<=loop['x2']]
        if len(x2) > 0:
            anchor1=True


        sameChromosome = peaks[peaks['Chromosome']==loop['chr2']]
        x1=sameChromosome[sameChromosome['Start']>=loop['y1']]
        x2=x1[x1['End']<=loop['y2']]
        if len(x2) > 0:
            anchor2=True

        if anchor1 == True and anchor2 == True:
            bothAnchorsIndexes.append(i)
            singleAnchorCount+=1
        elif anchor1 == True or anchor2 == True:
            singleAnchorCount+=1
    return bothAnchorsIndexes, singleAnchorCount

In [12]:
def applyGetAnchors(loop,peaks):
    anchor1=False
    anchor2=False
    sameChromosome = peaks[peaks['Chromosome']==loop['#chr1']]
    x1=sameChromosome[sameChromosome['Start']>=loop['x1']]
    x2=x1[x1['End']<=loop['x2']]
    if len(x2) > 0:
        anchor1=True
    sameChromosome = peaks[peaks['Chromosome']==loop['chr2']]
    x1=sameChromosome[sameChromosome['Start']>=loop['y1']]
    x2=x1[x1['End']<=loop['y2']]
    if len(x2) > 0:
        anchor2=True
    if anchor1 == True and anchor2 == True:
        return loop

In [14]:
peakLoopHicLCtcfPPairs,peakLoopHicLCtcfPSingleAnchorCount = getAnchors(hicLoops,ctcfPeaks)
with open("peakLoopHicLCtcfPPairs", "wb") as f:
    pickle.dump(peakLoopHicLCtcfPPairs, f)
with open("peakLoopHicLCtcfPSingleAnchorCount", "wb") as f:
    pickle.dump(peakLoopHicLCtcfPSingleAnchorCount, f)

In [63]:
peakLoopHicLRad21PPairs,peakLoopHicLRad21PSingleAnchorCount = getAnchors(hicLoops,rad21Peaks)
with open("peakLoopHicLRad21PPairs", "wb") as f:
    pickle.dump(peakLoopHicLRad21PPairs, f)
with open("peakLoopHicLRad21PSingleAnchorCount", "wb") as f:
    pickle.dump(peakLoopHicLRad21PSingleAnchorCount, f)

In [4]:
peakLoopChiaLCtcfPPairs,peakLoopChiaLCtcfPSingleAnchorCount = getAnchors(chiaPetLoops,ctcfPeaks)
with open("peakLoopChiaLCtcfPPairs", "wb") as f:
    pickle.dump(peakLoopChiaLCtcfPPairs, f)
with open("peakLoopChiaLCtcfPSingleAnchorCount", "wb") as f:
    pickle.dump(peakLoopChiaLCtcfPSingleAnchorCount, f)

In [5]:
peakLoopChiaLRad21PPairs,peakLoopChiaLRad21PSingleAnchorCount = getAnchors(chiaPetLoops,rad21Peaks)
with open("peakLoopChiaLRad21PPairs", "wb") as f:
    pickle.dump(peakLoopChiaLRad21PPairs, f)
with open("peakLoopChiaLRad21PSingleAnchorCount", "wb") as f:
    pickle.dump(peakLoopChiaLRad21PSingleAnchorCount, f)

Perform 4 comparisons between datasets. for two compared datasets (A and B) calculate how many loops from A match (**) some loop from B, and how many loops from B match some loop from A. You can present the result as a 2x2 table showing counts of total and matched loops for datasets.

In [16]:
with open(r"peakLoopChiaLCtcfPPairs", "rb") as input_file: # Chia LC
     peakLoopChiaLCtcfPPairs = pickle.load(input_file)

with open(r"peakLoopHicLCtcfPPairs", "rb") as input_file: # HIC LC
     peakLoopHicLCtcfPPairs = pickle.load(input_file)

with open(r"peakLoopChiaLRad21PPairs", "rb") as input_file: # CHIA Rad
     peakLoopChiaLRad21PPairs = pickle.load(input_file)

with open(r"peakLoopHicLRad21PPairs", "rb") as input_file: # Hic Rad
     peakLoopHicLRad21PPairs = pickle.load(input_file)

In [17]:
hicLcHicRadIndex = 0
singleAnchorHicLcHicRadIndex = 0
for hicLcIndex in peakLoopHicLCtcfPPairs:
     for hicRadIndex in peakLoopHicLRad21PPairs:
          if abs(hicLoops[hicLcIndex]['x1']-hicLoops[hicRadIndex]['x1'])<=1 and abs(hicLoops[hicLcIndex]['x2']-hicLoops[hicRadIndex]['x2'])<=1 and abs(hicLoops[hicLcIndex]['y1']-hicLoops[hicRadIndex]['y1'])<=1 and abs(hicLoops[hicLcIndex]['y2']-hicLoops[hicRadIndex]['y2'])<=1:
               hicLcHicRadIndex+=1
          # elif (abs(hicLoops[hicLcIndex]['x1']-hicLoops[hicRadIndex]['x1'])<=1 and abs(hicLoops[hicLcIndex]['x2']-hicLoops[hicRadIndex]['x2'])<=1) or(abs(hicLoops[hicLcIndex]['y1']-hicLoops[hicRadIndex]['y1'])<=1 and abs(hicLoops[hicLcIndex]['y2']-hicLoops[hicRadIndex]['y2'])<=1):
               # singleAnchorHicLcHicRadIndex+=1
print(hicLcHicRadIndex)
# print(singleAnchorHicLcHicRadIndex)

0
0


In [18]:
chiaLcChiaRadCount = 0
singleAnchorChiaLcChiaRadCount = 0
for chiaLcIndex in peakLoopChiaLCtcfPPairs:
     for chiaLRadIndex in peakLoopChiaLRad21PPairs:
          if abs(chiaPetLoops.iloc[chiaLcIndex]['x1']-chiaPetLoops.iloc[chiaLRadIndex]['x1'])<=1 and abs(chiaPetLoops.iloc[chiaLcIndex]['x2']-chiaPetLoops.iloc[chiaLRadIndex]['x2'])<=1 and abs(chiaPetLoops.iloc[chiaLcIndex]['y1']-chiaPetLoops.iloc[chiaLRadIndex]['y1'])<=1 and abs(chiaPetLoops.iloc[chiaLcIndex]['y2']-chiaPetLoops.iloc[chiaLRadIndex]['y2'])<=1:
               chiaLcChiaRadCount+=1
          # elif abs(chiaPetLoops.iloc[chiaLcIndex]['x1']-chiaPetLoops.iloc[chiaLRadIndex]['x1'])<=1 or abs(chiaPetLoops.iloc[chiaLcIndex]['x2']-chiaPetLoops.iloc[chiaLRadIndex]['x2'])<=1:
            #   singleAnchorChiaLcChiaRadCount+=1
print(chiaLcChiaRadCount)
# print(singleAnchorChiaLcChiaRadCount)

7059


In [22]:
with open(r"chiaLcChiaRadCount", "wb") as input_file:
     pickle.dump(chiaLcChiaRadCount,input_file)

In [24]:
chiaLcHicLcCount = 0
singleAnchorChiaLcHicLcCount = 0
for chiaLcIndex in peakLoopChiaLCtcfPPairs:
     for hicLcIndex in peakLoopHicLCtcfPPairs:
          if (abs(chiaPetLoops.iloc[chiaLcIndex]['x1']-hicLoops.iloc[hicLcIndex]['x1'])<=1 and abs(chiaPetLoops.iloc[chiaLcIndex]['x2']-hicLoops.iloc[hicLcIndex]['x2'])<=1) and (abs(chiaPetLoops.iloc[chiaLcIndex]['y1']-hicLoops.iloc[hicLcIndex]['y1'])<=1 and abs(chiaPetLoops.iloc[chiaLcIndex]['y2']-hicLoops.iloc[hicLcIndex]['y2'])<=1):
               chiaLcHicLcCount+=1
          # if abs(chiaPetLoops.iloc[chiaLcIndex[0]][0]-hicLoops.iloc[hicLcIndex[0]][0])<=1 or abs(chiaPetLoops.iloc[chiaLcIndex[0]][1]-hicLoops.iloc[hicLcIndex[0]][1])<=1:
            #    singleAnchorChiaLcHicLcCount+=1
print(chiaLcHicLcCount)
# print(singleAnchorChiaLcHicLcCount)

0


In [25]:
with open(r"chiaLcHicLcCount", "wb") as input_file:
     pickle.dump(chiaLcHicLcCount,input_file)

In [28]:
chiaLcRadHicCtcfCount = 0
singleAnchorChiaLcHicRadCount = 0
for chiaLcIndex in peakLoopChiaLRad21PPairs:
     for hicRadIndex in peakLoopHicLCtcfPPairs:
          if (abs(chiaPetLoops.iloc[chiaLcIndex]['x1']-hicLoops.iloc[hicRadIndex]['x1'])<=1 and abs(chiaPetLoops.iloc[chiaLcIndex]['x2']-hicLoops.iloc[hicRadIndex]['x2'])<=1) and (abs(chiaPetLoops.iloc[chiaLcIndex]['y1']-hicLoops.iloc[hicRadIndex]['y1'])<=1 and abs(chiaPetLoops.iloc[chiaLcIndex]['y2']-hicLoops.iloc[hicRadIndex]['y2'])<=1):
               chiaLcRadHicCtcfCount+=1
  #        if abs(chiaPetLoopsXArr[chiaLcIndex[0]][0]-hicLoopsXArr[hicRadIndex[0]][0])<=1 or abs(chiaPetLoopsXArr[chiaLcIndex[0]][1]-hicLoopsXArr[hicRadIndex[0]][1])<=1:
   #            singleAnchorChiaLcHicRadCount+=1
print(chiaLcRadHicCtcfCount)
# print(singleAnchorChiaLcHicRadCount)

In [ ]:
with open(r"chiaLcRadHicCtcfCount", "wb") as input_file:
     pickle.dump(chiaLcRadHicCtcfCount,input_file)

Make a visualization of a selected region (e.g. in IGV), choose something that looks nicely, zoom in enough so that you can see the peak locations. You can also include the loops matched between 2 datasets.